In [1]:
%load_ext lab_black

In [200]:
import json
import gzip
import pandas as pd
import glob
import tempfile
from importlib import reload

import uuid
import os
from pprint import pprint
from multiprocessing import Pool
from pathlib import Path
from sadie.airr import Airr, AirrTable
from gspread_pandas import Spread, Client
from sadie.airr import AirrTable
from sadie.airr import constants
from distutils.version import StrictVersion
from math import nan
from IPython.display import clear_output
from numpy import isnan
from IPython.core.display import display, HTML
from sadie.antibody import AntibodyChainNT


spread = Spread("Sadie", create_spread=True, create_sheet=True)
pd.set_option("display.max_rows", 45)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 100000)


def fillna(df, fill_value=""):
    """
    Replace null values with `fill_value`.

    Also replaces in categorical columns.
    """
    for col in df.dtypes[df.dtypes == "category"].index:
        if fill_value not in df[col].cat.categories:
            df[col].cat.add_categories([fill_value], inplace=True)
    # Known bug https://github.com/pandas-dev/pandas/issues/25472
    if StrictVersion(pd.__version__) >= StrictVersion("1.0"):
        for col in df.dtypes[
            df.dtypes.apply(lambda x: x in ["float64", "Int16", "Int64"])
        ].index:
            df[col] = df[col].astype("float")
    return df.fillna(fill_value)

# Sadie Annotate

In [109]:
# straight ignore these
ignore = [
    #     "v_call",
    #     "d_call",
    #     "j_call",
    "v_score",
    "d_score",
    "j_score",
    "v_support",
    "d_support",
    "j_support",
]

# cast these to integers
starts_and_ends = [
    "cdr1_end",
    "cdr1_start",
    "cdr2_end",
    "cdr2_start",
    "cdr3_end",
    "cdr3_start",
    "d_alignment_end",
    "d_alignment_start",
    "d_germline_end",
    "d_germline_start",
    "d_sequence_end",
    "d_sequence_start",
    "fwr1_end",
    "fwr1_start",
    "fwr2_end",
    "fwr2_start",
    "fwr4_end",
    "fwr4_start",
    "j_alignment_end",
    "j_alignment_start",
    "j_germline_end",
    "j_germline_start",
    "j_sequence_end",
    "j_sequence_start",
]


def make_sadie_comparable(df):
    if isinstance(df, AirrTable):
        df = df.table
    # comparison keys between imgt and sadie
    compare_key = list(constants.IGBLAST_AIRR.keys()) + [
        "v_call_top",
        "d_call_top",
        "j_call_top",
    ]
    compare_key.remove("v_frameshift")
    df = df[compare_key].drop(ignore, axis=1)
    df.loc[:, starts_and_ends] = df[starts_and_ends].astype("Int64")
    df.insert(
        df.columns.get_loc("v_call_top"),
        "v_gene_top",
        df["v_call_top"].str.split("*").str.get(0),
    )
    df.insert(
        df.columns.get_loc("d_call_top"),
        "d_gene_top",
        df["d_call_top"].str.split("*").str.get(0),
    )
    df.insert(
        df.columns.get_loc("j_call_top"),
        "j_gene_top",
        df["j_call_top"].str.split("*").str.get(0),
    )
    return df


def make_imgt_comparable(df):
    def map_bool(mapp_df, col):
        return mapp_df.loc[:, col].map(
            {"T": True, "F": False, True: True, False: False, nan: False},
            na_action="ignore",
        )

    compare_key = list(constants.IGBLAST_AIRR.keys())
    compare_key.remove("v_frameshift")
    df = df[compare_key].copy()
    bool_cols = ["vj_in_frame", "productive", "rev_comp", "complete_vdj", "stop_codon"]
    for col in bool_cols:
        df[col] = map_bool(df, col)

    upper_columns = [
        "fwr1",
        "cdr1",
        "fwr2",
        "cdr2",
        "fwr3",
        "cdr3",
        "fwr4",
        "germline_alignment",
        "germline_alignment_aa",
        "sequence_alignment",
        "sequence_alignment_aa",
        "v_germline_alignment",
        "v_germline_alignment_aa",
        "v_sequence_alignment",
        "v_sequence_alignment_aa",
        "d_germline_alignment",
        "d_germline_alignment_aa",
        "d_sequence_alignment",
        "d_sequence_alignment_aa",
        "j_germline_alignment",
        "j_germline_alignment_aa",
        "j_sequence_alignment",
        "j_sequence_alignment_aa",
        "np1",
        "np2",
        "junction",
        "sequence",
    ]
    for col in upper_columns:
        df[col] = df[col].str.upper()

    df["fwr1"] = df["fwr1"].str.lstrip(".")
    df["fwr4"] = df["fwr4"].str.rstrip(".")
    df["v_call_top"] = df["v_call"].str.split(",").str.get(0).str.split().str.get(1)
    df["v_gene_top"] = df["v_call_top"].str.split("*").str.get(0)
    df["d_call_top"] = df["d_call"].str.split(",").str.get(0).str.split("_").str.get(-1)
    df["d_gene_top"] = df["d_call_top"].str.split("*").str.get(0)
    df["j_call_top"] = df["j_call"].str.split(",").str.get(0).str.split().str.get(1)
    df["j_gene_top"] = df["j_call_top"].str.split("*").str.get(0)
    df = df.drop(ignore, axis=1)
    df.loc[:, starts_and_ends] = df[starts_and_ends].astype("Int64")
    return df

### Sadie

Run sadie against the OAS subsample and good full length anarci

In [37]:
sub_sample_file_good_anarci = (
    "../../tests/integration/airr/fixtures/OAS_subsample_good_anarci.fasta"
)
airr_api = Airr(species="human", database="imgt", functional="all")
# airr_api.igblast.v_penalty = -1
# airr_api.igblast.j_penalty = -1

In [242]:
airr_api = Airr(species="human", database="imgt", functional="all")
airr_api.igblast.v_penalty = -1
airr_api.igblast.j_penalty = -2
airr_api.run_single(
    "test",
    "GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCAGGCGAGTCAGGACATTAGCAACTATTTAAATTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAGCTCCTGATCTACGATGCATCCAATTTGGAAACAGGGGTCCCATCAAGGTTCAGTGGAAGTGGATCTGGGACAGATTTTACTTTCACCATCAGCAGCCTGCAGCCTGAAGATATTGCAACATATTACTGTCAACAGTATGATAATTTCGGCGGAGGGACCAAGGTGGACATCAAAC",
)

,sequence_id,sequence,species,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call_top,v_call,d_call_top,d_call,j_call_top,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,fwr4,fwr4_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,fwr4_start,fwr4_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,note,vdj_nt,vdj_aa
0,test,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,human,IGK,False,True,False,True,False,True,IGKV1-33*01,"IGKV1-33*01,IGKV1D-33*01",NaN,NaN,IGKJ4*01,"IGKJ4*01,IGKJ4*02",GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,DIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKL...,DIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKL...,1,282,<NA>,<NA>,283,310,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,DIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKL...,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,DIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKL...,NaN,NaN,NaN,NaN,GGCGGAGGGACCAAGGTGGACATCAAAC,GGGTKVDIK,GGCGGAGGGACCAAGGTGGAGATCAAAC,GGGTKVEIK,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,DIQMTQSPSSLSASVGDRVTITCQAS,CAGGACATTAGCAACTAT,QDISNY,TTAAATTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAGCTCCTGA...,LNWYQQKPGKAPKLLIY,GATGCATCC,DAS,AATTTGGAAACAGGGGTCCCATCAAGGTTCAGTGGAAGTGGATCTG...,NLETGVPSRFSGSGSGTDFTFTISSLQPEDIATYYC,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,438.0,NaN,48.75,282M28S5N,NaN,282S10N28M,5.992000e-125,NaN,6.987000e-10,99.625,NaN,96.4375,1,282,1,282,<NA>,<NA>,NaN,NaN,283,310,11,38,1,78,79,96,97,147,148,156,157,264,<NA>,<NA>,<NA>,<NA>,NaN,0,NaN,<NA>,potential broken,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAG...,DIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKL...


In [38]:
sadie_airr_good_anarci = airr_api.run_file(sub_sample_file_good_anarci)
sadie_comparable = make_sadie_comparable(sadie_airr_good_anarci)

#### IMGT HiVquest for the same FASTA

We have to run this at their website

In [110]:
imgt_airr = (
    "../../tests/integration/airr/fixtures/OAS_airr_from_imgtvquest_fl_anarci_good.tsv"
)
imgt_df = pd.read_csv(imgt_airr, delimiter="\t", low_memory=False)
imgt_comparable = make_imgt_comparable(imgt_df)

#### Dump to feather just incase we get DC'd

In [111]:
sadie_comparable.to_feather("sadie_compariable_good.feather")
imgt_comparable.to_feather("imgt_compariable_good.feather")

# Compare

#### Reload Comparison

In [112]:
sadie_comparable = pd.read_feather("sadie_compariable_good.feather")
imgt_comparable = pd.read_feather("imgt_compariable_good.feather")

In [113]:
# The fields we can compare
check_these = [
    "sequence",
    "locus",
    "stop_codon",
    "vj_in_frame",
    "productive",
    "rev_comp",
    "complete_vdj",
    "v_call",
    "v_call_top",
    "v_gene_top",
    "d_call_top",
    "d_gene_top",
    "j_call_top",
    "j_gene_top",
    "fwr1",
    "cdr1",
    "fwr2",
    "cdr2",
    "fwr3",
    "cdr3",
    "fwr1_aa",
    "cdr1_aa",
    "fwr2_aa",
    "cdr2_aa",
    "fwr3_aa",
    "cdr3_aa",
    "fwr4_aa",
]

### Stop Codon Corrections


This is where the imgt did not agree with the sadie on stop codon

In [231]:
def comparable(compare_field):
    compare = imgt_comparable[check_these] == sadie_comparable[check_these]
    disagreements_index = compare[~compare[compare_field]].index

    disagree_imgt = imgt_comparable.loc[disagreements_index]
    disagree_sadie = sadie_comparable.loc[disagreements_index]

    def render_ab_object(row):
        # print(row['v_call_top'],row['j_call_top'],row['fwr1'],row['cdr1'],row['fwr2'],row['cdr2'],row['fwr3'],row['cdr3'],row['fwr4'],'\n')
        return AntibodyChainNT(
            name=row["sequence_id"],
            fwr1_nt=row["fwr1"],
            cdr1_nt=row["cdr1"],
            fwr2_nt=row["fwr2"],
            cdr2_nt=row["cdr2"],
            fwr3_nt=row["fwr3"],
            cdr3_nt=row["cdr3"],
            fwr4_nt=row["fwr4"],
            v_gene=row["v_call_top"],
            j_gene=row["j_call_top"],
            species="human",
        )

    print(f"{len(disagreements_index)} disagreements")
    imgt_df_index = imgt_df.set_index("sequence_id")
    for i in disagreements_index:
        rendered_string = ""
        imgt_row = imgt_comparable.loc[i].fillna("")
        if imgt_row["sequence_id"] in list(
            map(
                lambda x: x["sequence_id"],
                json.load(
                    open("../../tests/integration/airr/fixtures/imgt_exceptions.json")
                ),
            )
        ):
            print(imgt_row["sequence_id"], "accounted for")
            continue
        insertions = imgt_df_index.loc[imgt_row["sequence_id"], "insertions"]
        deletions = imgt_df_index.loc[imgt_row["sequence_id"], "deletions"]

        if isinstance(insertions, str):
            if compare_field.upper() in insertions:
                print("insertion not accounted for")
                continue
        #         print("insertions", insertions)
        #         print("deletions", deletions)
        imgt_object = render_ab_object(imgt_row)
        saide_row = sadie_comparable.loc[i].fillna("")
        sadie_object = render_ab_object(saide_row)
        rendered_string += f"IMGT\n{imgt_object.get_segmented_alignment_nt()}"
        rendered_string += f"SADIE\n{sadie_object.get_segmented_alignment_nt()}"
        rendered_string += f"IMGT\n{imgt_object.get_segmented_alignment_aa()}"
        rendered_string += f"SADIE\n{sadie_object.get_segmented_alignment_aa()}"

        print(saide_row["sequence_id"])
        print(imgt_row["sequence_id"])
        print(imgt_row[compare_field])
        print(saide_row[compare_field])

        print(saide_row["sequence"])
        print(rendered_string)
        input()
        clear_output()


comparable("vj_in_frame")

In [222]:
sadie_comparable.set_index("sequence_id").loc[
    "150_Donor03+IGH-Clonotypes_igblastn_anarci_Heavy_Bulk_8491608", "complete_vdj"
]

False

In [223]:
imgt_df.set_index("sequence_id").loc[
    "3795_Donor03+IGH-Clonotypes_igblastn_anarci_Heavy_", "complete_vdj"
]

'T'

In [163]:
spread.df_to_sheet(
    imgt_df.set_index("sequence_id")
    .loc[
        "456_Donor02+IGH-Clonotypes_igblastn_anarci_Heavy_B",
    ]
    .to_frame()
)

In [135]:
"CAGGTGCAACTGGTGGCGTCTGGGGGAGGCGTGGTCCGGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCT"

'CAGGTGCAACTGGTGGCGTCTGGGGGAGGCGTGGTCCGGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCT'

In [218]:
airr_api.igblast.v_penalty = -1
airr_api.run_single(
    "test",
    "GATCCTCTTCTCATAGTGCCTCCATCAGAGCATGGCTGTCCTGGTGCTGCTCCTCTGCCTGGTGACATTCCCAAGCTGTGTCCTATCCCAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCACAGAGCCTGTCCATAACCTGCACAGTCTCTGGCGTCTCATTAAGTGACTATGGTGTCCACTGGGTTCGCCAGTCTCCAGGAAAGGGCCTGGAGTGGCTGGGAGTGATATGGAGAGGTGGAGGCACAGACTACAGTGCAGCTTTCATGTCCAGAATGAGGCTCACCAAGGACAATTCCAAGAGTCAAGTTTTCTTTGAACTGAACCGTCTGCACGATGATGACACTGCCGTATACTACTGTATCAAAGAAACCTACTATAGTTACGAGAGCTACTTCTACTTCGATGTCTGGGGCACAGGGACCACGGTCACCGTCTCCTCAGAGAGTCAGTCCTTCCCAAATGTCTTCCC",
)

,sequence_id,sequence,species,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call_top,v_call,d_call_top,d_call,j_call_top,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,fwr4,fwr4_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,fwr4_start,fwr4_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,vdj_nt,vdj_aa
0,test,GATCCTCTTCTCATAGTGCCTCCATCAGAGCATGGCTGTCCTGGTG...,human,IGH,False,True,False,True,False,True,IGHV3-30*18,"IGHV3-30*18,IGHV3-30-5*01,IGHV3-33*06",IGHD3-10*01,"IGHD3-10*01,IGHD5-18*01,IGHD5-5*01",IGHJ6*03,IGHJ6*03,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,QVQLKQSGPGLVLPSQSLSITCTVSGVSLSDYGVHWVRQSPGKGLE...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,1,297,302,311,319,370,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,QVQLKQSGPGLVLPSQSLSITCTVSGVSLSDYGVHWVRQSPGKGLE...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,TACTATAGTT,YYS,TACTATGGTT,YYG,CTACTTCTACTTCGATGTCTGGGGCACAGGGACCACGGTCACCGTC...,YFYFDVWGTGTTVTVSS,CTACTACTACATGGACGTCTGGGGCAAAGGGACCACGGTCACCGTC...,YYYMDVWGKGTTVTVSS,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,QVQLKQSGPGLVLPSQSLSITCTVS,GGCGTCTCATTAAGTGACTATGGT,GVSLSDYG,GTCCACTGGGTTCGCCAGTCTCCAGGAAAGGGCCTGGAGTGGCTGG...,VHWVRQSPGKGLEWLGV,ATATGGAGAGGTGGAGGCACAGAC,IWRGGGTD,TACAGTGCAGCTTTCATGTCCAGAATGAGGCTCACCAAGGACAATT...,YSAAFMSRMRLTKDNSKSQVFFELNRLHDDDTAVYYC,TGGGGCACAGGGACCACGGTCACCGTCTCCTCA,WGTGTTVTVSS,ATCAAAGAAACCTACTATAGTTACGAGAGCTACTTCTACTTCGATGTC,IKETYYSYESYFYFDV,TGTATCAAAGAAACCTACTATAGTTACGAGAGCTACTTCTACTTCG...,54,CIKETYYSYESYFYFDVW,18,131.125,14.148438,71.8125,88S184M3D42M1I14M1D52M102S,385S4N10M88S17N,402S10N52M29S,2.417000e-32,16.44,1.210000e-16,66.0,90.0,90.375,89,381,1,296,386,395,5.0,14.0,403,454,11,62,89,163,164,187,188,238,239,262,263,373,422,454,374,421,AACC,4,ACGAGAG,7,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,QVQLKQSGPGLVLPSQSLSITCTVSGVSLSDYGVHWVRQSPGKGLE...


In [216]:
test_object

,sequence_id,sequence,species,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call_top,v_call,d_call_top,d_call,j_call_top,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,fwr4,fwr4_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,fwr4_start,fwr4_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,vdj_nt,vdj_aa
0,test,GATCCTCTTCTCATAGTGCCTCCATCAGAGCATGGCTGTCCTGGTG...,human,IGH,False,True,False,True,False,True,IGHV3-30*18,"IGHV3-30*18,IGHV3-30-5*01,IGHV3-33*06",IGHD3-10*01,"IGHD3-10*01,IGHD5-18*01,IGHD5-5*01",IGHJ6*03,IGHJ6*03,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,QVQLKQSGPGLVLPSQSLSITCTVSGVSLSDYGVHWVRQSPGKGLE...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,1,297,302,311,319,370,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,QVQLKQSGPGLVLPSQSLSITCTVSGVSLSDYGVHWVRQSPGKGLE...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,TACTATAGTT,YYS,TACTATGGTT,YYG,CTACTTCTACTTCGATGTCTGGGGCACAGGGACCACGGTCACCGTC...,YFYFDVWGTGTTVTVSS,CTACTACTACATGGACGTCTGGGGCAAAGGGACCACGGTCACCGTC...,YYYMDVWGKGTTVTVSS,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,QVQLKQSGPGLVLPSQSLSITCTVS,GGCGTCTCATTAAGTGACTATGGT,GVSLSDYG,GTCCACTGGGTTCGCCAGTCTCCAGGAAAGGGCCTGGAGTGGCTGG...,VHWVRQSPGKGLEWLGV,ATATGGAGAGGTGGAGGCACAGAC,IWRGGGTD,TACAGTGCAGCTTTCATGTCCAGAATGAGGCTCACCAAGGACAATT...,YSAAFMSRMRLTKDNSKSQVFFELNRLHDDDTAVYYC,TGGGGCACAGGGACCACGGTCACCGTCTCCTCA,WGTGTTVTVSS,ATCAAAGAAACCTACTATAGTTACGAGAGCTACTTCTACTTCGATGTC,IKETYYSYESYFYFDV,TGTATCAAAGAAACCTACTATAGTTACGAGAGCTACTTCTACTTCG...,54,CIKETYYSYESYFYFDVW,18,131.125,14.148438,71.8125,88S184M3D42M1I14M1D52M102S,385S4N10M88S17N,402S10N52M29S,2.417000e-32,16.44,1.210000e-16,66.0,90.0,90.375,89,381,1,296,386,395,5.0,14.0,403,454,11,62,89,163,164,187,188,238,239,262,263,373,422,454,374,421,AACC,4,ACGAGAG,7,CAGGTGCAGCTGAAGCAGTCAGGACCTGGCCTAGTGCTGCCCTCAC...,QVQLKQSGPGLVLPSQSLSITCTVSGVSLSDYGVHWVRQSPGKGLE...


In [146]:
test_object

,sequence_id,sequence,species,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call_top,v_call,d_call_top,d_call,j_call_top,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,fwr4,fwr4_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,fwr4_start,fwr4_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,vdj_nt,vdj_aa
0,test,CAGCTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGTCCGGGA...,human,IGH,False,True,False,True,False,True,IGHV3-30-3*01,IGHV3-30-3*01,IGHD3-22*01,IGHD3-22*01,IGHJ4*02,IGHJ4*02,CAGCTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGTCCGGGA...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,QLQLVESGGGVVQSGRSLRLSCAVSAASGYSFSNYAIHWARQAPGK...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYAMHWVRQAPGKGLE...,1,303,323,336,350,391,CAGCTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGTCCGGGA...,QLQLVESGGGVVQSGRSLRLSCAVSAASGYSFSNYAIHWARQAPGK...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGA...,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYAMHWVRQAPGKGLE...,ATGATAGTAGCGGT,DSSG,ATGATAGTAGTGGT,DSSG,TTGGCTCCTGGGGCCAGGGAACCCTGGTGTCCGTCTCCTCAG,GSWGQGTLVSVSS,TTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,DYWGQGTLVTVSS,CAGCTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGTCCGGGA...,QLQLVESGGGVVQSGRSLRLSCAVSAAS,GGATACAGCTTCAGTAACTACGCT,GYSFSNYA,ATTCACTGGGCCCGCCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGG...,IHWARQAPGKGLEWVAV,ATATCATATGATGGGAACACTAAA,ISYDGNTK,TACTACGCAGACTCCGTGACGGGCCGATTCACCATCTCCAGAGACA...,YYADSVTGRFTISRDNSKGTLYLQMNSLKPEDTAMYYC,TGGGGCCAGGGAACCCTGGTGTCCGTCTCCTCA,WGQGTLVSVSS,GCGAGACTTTCGTTCCGACATTTCCATGATAGTAGCGGTCCCCAAA...,ARLSFRHFHDSSGPQISVGS,TGTGCGAGACTTTCGTTCCGACATTTCCATGATAGTAGCGGTCCCC...,66,CARLSFRHFHDSSGPQISVGSW,22,368.0,21.84375,58.375,70M9I224M88S2N,322S8N14M55S9N,349S6N42M,9.862000e-104,0.06408,1.097000e-12,89.4375,92.875,90.5,1,303,1,294,323,336,9.0,22.0,350,391,7,48,1,84,85,108,109,159,160,183,184,297,358,390,298,357,CTTTCGTTCCGACATTTCC,19,CCCCAAATTTCTG,13,CAGCTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGTCCGGGA...,QLQLVESGGGVVQSGRSLRLSCAVSAASGYSFSNYAIHWARQAPGK...


In [28]:
sadie_object

'GCAAGGCCAGTCAGAATGTGGGTACTAATGTAGCCTGGTATCAACAGAAACCAGGGCAATCTCCTAAAGCACTGATTTACTCGGCATTGTACCGATACAGTGGAGTCCCTGATCGCTTCACAGGCAGTGGATCTGGGACAGATTTCACTCTCACCATCAGCAATGTGCAGTCTGAAGACCTGGCAGTTTATTTCTGTCAGCAGGATTATAGCTCTCCGTACACGTTCGGAGGGGGGACCAAGCTGGAAATAAAA'

In [9]:
imgt_object

2125_Subject-15_igblastn_anarci_Heavy_IGHA_162
FrameWork1NT 1-0:
CDR1NT 1-0:
FrameWork2NT 1-33:CTGGGGGGCCCTTAAGACCCCCCTNTCACCCAC
CDR2NT 34-57:CATATCAGGTTTCGGCGCCTTTCC
FrameWork3NT 58-171:TATGAACTGGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAACTCCAAGGACACGGTGTATCTGCAAATGGACAGCCTGAGACCTGAGGACACGGCTGTGTACTACTGT
CDR3NT 172-207:GCGAGGACAGTTGCAGTGGGTTGGTACTGTGACTAC
FrameWork4NT 208-241:TGGGGCCAGGGAACCCAGGTCACCGTCTCCTCAC

In [95]:
print(imgt_mab.get_segmented_alignment_nt())

IGHV5-51*01|IGHJ4*01  GAGGTGCAGCTGGTGCAGTCTGGAGCAGAGGTGAAAAAGCCCGGGGAGTCTCTGAAGATCTCCTGTAAGGGTTCT GGATACAGCTTTACCAGCTACTGG
somecrap              ACAC.TTT.TAAAAT.-------------------------------T.GGGG..................C... .......C....C...AT......

IGHV5-51*01|IGHJ4*01   ATCGGCTGGGTGCGCCAGATGCCCGGGAAAGGCCTGGAGTGGATGGGGATC ATCTATCCTGGTGACTCTGATACC AGATACAGCCCGTCCTTCCAAG
somecrap               ................................................G.. .....C....A..........G.. ........T........TG.G.

IGHV5-51*01|IGHJ4*01  GCCAGGTCACCATCTCAGCCGACAAGTCCATCAGCACCGCCTACCTGCAGTGGAGCAGCCTGAAGGCCTCGGACACCGCCATGTATTACTGT GCGAGAC
somecrap              .T...............................C...............A.....G................................T... .......

IGHV5-51*01|IGHJ4*01  A-----------ACTACTTTGACTAC TGGGGCCAAGGAACCCTGGTCACCGTCTCCTCAG
somecrap              GGAGAGATGGCT..A........... ........G.A.G............C......G.




In [74]:
single_entry = imgt_comparable.loc[i]

In [84]:
single_entry["j_gene_top"]

'IGHJ4'

In [167]:
# you cannot trust fwr4 since it has the extra nucleotide in the jcall

In [172]:
pd.set_option("display.max_rows", 100)
for i in range(6,20):
    single_entry_diff = (
        imgt_comparable.loc[compare[~compare["v_gene_top"]].index]
        .iloc[i]
        .to_frame()
        .join(
            sadie_comparable.loc[compare[~compare["v_gene_top"]].index].iloc[i].to_frame(),
            rsuffix="_sadie",
            lsuffix="_imgt",
        )
        .join(compare[~compare["v_gene_top"]].iloc[i].to_frame())
        .fillna("")
    ).loc[check_these]
    print(imgt_comparable.loc[compare[~compare["v_gene_top"]].index].iloc[i]['sequence'])
    display(HTML(single_entry_diff._repr_html_()))
    input()
    clear_output()

CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTCAGTTATTTTGGCATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACTGGAGTGGGTGGCACTGATAAAGTATGATGGAAGTAATACATACCACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACACTATATTTGCAAATGAACAGCCTGAGACCCGAGGACACGGCCATATATTACTGTGCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCGGCTGCTTTGACTCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCGCCCTGCTCCAGGAGCACCTCTGGGGGCACAGCGGCCCTGGGCTGCCTG


,431_imgt,431_sadie,431
sequence,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,True
locus,IGH,IGH,True
stop_codon,True,False,False
vj_in_frame,True,True,True
productive,True,True,True
rev_comp,False,False,True
complete_vdj,False,False,True
v_call,"Homsap IGHV3-30*02 F, or Homsap IGHV3-30-5*02 ...",IGHV3-33*06,False
v_call_top,IGHV3-30*02,IGHV3-33*06,False
v_gene_top,IGHV3-30,IGHV3-33,False


KeyboardInterrupt: Interrupted by user

In [175]:
airr_api.run_single(
    "test",
    "CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTCAGTTATTTTGGCATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACTGGAGTGGGTGGCACTGATAAAGTATGATGGAAGTAATACATACCACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACACTATATTTGCAAATGAACAGCCTGAGACCCGAGGACACGGCCATATATTACTGTGCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCGGCTGCTTTGACTCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCGCCCTGCTCCAGGAGCACCTCTGGGGGCACAGCGGCCCTGGGCTGCCTG",
)

,sequence_id,sequence,species,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call_top,v_call,d_call_top,d_call,j_call_top,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,fwr4,fwr4_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,fwr4_start,fwr4_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,vdj_nt,vdj_aa
0,test,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,human,IGH,False,True,False,True,False,False,IGHV3-33*06,IGHV3-33*06,IGHD5-24*01,IGHD5-24*01,IGHJ4*02,IGHJ4*02,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,TGGGAGGTCCCTGAGACTCTCCTGTGCAGCGTCTGGATTCACCTTC...,GESLKISCAASGFTFSYFGMHWVRQAPGKGLEWVALIKYDGSNTYH...,GRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYDGSNKYY...,1,255,269,279,292,338,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,GESLKISCAASGFTFSYFGMHWVRQAPGKGLEWVALIKYDGSNTYH...,TGGGAGGTCCCTGAGACTCTCCTGTGCAGCGTCTGGATTCACCTTC...,GRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYDGSNKYY...,TGGCTACAATT,WLQ,TGGCTACAATT,WLQ,CTGCTTTGACTCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,CFDSWGQGTLVTVSS,CTACTTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,YFDYWGQGTLVTVSS,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCT,GESLKISCAAS,GGATTCACGTTCAGTTATTTTGGC,GFTFSYFG,ATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACTGGAGTGGGTGG...,MHWVRQAPGKGLEWVAL,ATAAAGTATGATGGAAGTAATACA,IKYDGSNT,TACCACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACA...,YHADSVKGRFTISRDNSKNTLYLQMNSLRPEDTAIYYC,TGGGGCCAGGGAACCCTGGTCACCGTCTCCTCA,WGQGTLVTVSS,GCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCGGCT...,AKEMDMRWLQFHDFGCFDS,TGTGCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCG...,63,CAKEMDMRWLQFHDFGCFDSW,21,343.0,21.84375,69.75,41N255M166S,268S7N11M142S2N,291S1N47M83S,4.067000e-96,0.06912,3.339000e-16,89.4375,100.0,95.75,1,255,42,296,269,279,8.0,18.0,292,338,2,48,1,34,35,58,59,109,110,133,134,247,305,337,248,304,GATGGATATGCGG,13,TCACGATTTCGG,12,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,GESLKISCAASGFTFSYFGMHWVRQAPGKGLEWVALIKYDGSNTYH...


In [176]:
airr_api

IgBLAST: env IGDATA=/Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/bin/darwin/igblastn -min_D_match 5 -num_alignments_V 5 -num_alignments_J 3 -num_alignments_D 3 -organism human -ig_seqtype Ig -germline_db_V /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig/blastdb/human_V -germline_db_D /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig/blastdb/human_D -germline_db_J /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig/blastdb/human_J -auxiliary_data /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/aux_db/human_gl.aux -outfmt 19 -domain_system imgt -word_size 11 -gapopen 5 -gapextend 2 -J_penalty -1 -V_penalty -2 -num_threads 12 -show_translation -extend_align5end -extend_align3end

In [112]:
imgt_comparable.loc[55, "sequence"]

'CTTCGGAGACCCTGTCCCTCACCTGCGCTGTCTATGGTGGGTCCTTCAGTGGTTACTACTGGGGCTGGATCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATTATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCCGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCTGTGTATTACTGTGCGAACGGCCCTATGTATAGTGGGAGCTACGGGTGGTTCGACCCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGTAAG'

## Read OAS

In [22]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

sub_sample_file = "../../tests/integration/airr/fixtures/OAS_subsample.fasta"
with open("../../tests/integration/airr/fixtures/OAS_subsample.fasta", "w") as f:
    for index, (i, k, j) in enumerate(
        zip(
            df_bz2["OAS_dataunit"].str.split(".csv.gz").str.get(0),
            df_bz2["OAS_dataindex"],
            df_bz2["sequence"],
        )
    ):
        f.write(f">{index}_{i}_{k}\n{j}\n")

In [32]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample_good_anarci.csv.bz2",
    index_col=0,
).reset_index()

sub_sample_file = (
    "../../tests/integration/airr/fixtures/OAS_subsample_good_anarci.fasta"
)
with open(
    "../../tests/integration/airr/fixtures/OAS_subsample_good_anarci.fasta", "w"
) as f:
    for index, (i, k, j) in enumerate(
        zip(
            df_bz2["OAS_dataunit"].str.split(".csv.gz").str.get(0),
            df_bz2["OAS_dataindex"],
            df_bz2["sequence"],
        )
    ):
        f.write(f">{index}_{i}_{k}\n{j}\n")

/Users/jordanwillis/miniconda3/envs/sadie/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27,43,44,45,46,71,105) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## IMGT HI-Vqeust "AIRR" format

In [17]:
spread.df_to_sheet(
    imgt_df.head(100).set_index("sequence_id"),
    sheet="imgt",
    replace=True,
    freeze_headers=True,
    freeze_index=True,
)

In [ ]:
# spreadable_df = fillna(sadie_airr.table.head(100))
spread.df_to_sheet(
    spreadable_df.set_index("sequence_id"),
    sheet="sadie",
    replace=True,
    freeze_headers=True,
    freeze_index=True,
)

## Clean UP IMGT

## Clean up sadie

In [358]:
ref[check_these] == target[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2    False
fwr3     True
cdr3     True
Name: 99, dtype: bool

In [362]:
ref["cdr2_aa"]

'ISGSRTYI'

In [363]:
target["cdr2_aa"]

'ISGSFRTYI'

In [361]:
ref["sequence"]

'TACCTTGACTTAACTCTTGGGGGTCTGGGGGAGGCCTGGTCAAGCCTGGGGGGTCCCTGAGACTCTCATGTGCAGCCTCTGGATTCACCTTCAGTAGTTTTACCATGAATTGGGTCCGCCAGGCTCCAGGGAAGGGACTGGAGTGGGTCTCATCCATTAGCGGCAGTTTCAGAACTTACATATATTATGCCGACTCAGTGAAGGGCCGATTCACCGTCTCCAGAGACAACGCCAAGGAATTGGTGTTTCTGCAGATGGACAACCTGAGAGTCGAAGACACAGGTGTATATTATTGTGCGAGAGACCTAAATACGGTGACTACCCCAGAATACTTCCAACACTGGGGCCTGGGCACCCCGGTCTCCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTG'

In [355]:
target["cdr2"]

'ATCAACCCTAGTGGTGATAACACA'

In [344]:
ref["fwr4"]

'TGGGGCCAGGGAACC'

In [345]:

target["fwr4"]

'TGGGGCCAGGGAACCCT'

In [342]:
ref[check_these] == target[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2     True
fwr3     True
cdr3     True
fwr4    False
Name: 3, dtype: bool

In [328]:
len(ref["fwr4"])

33

In [320]:
ref["v_call_top"]

'IGHV4-31*02'

In [321]:
target["v_call_top"]

'IGHV4-31*02'

In [303]:
ref["j_call_top"]

'IGHJ4*02'

In [304]:
target["j_call_top"]

'IGHJ4*02'

In [330]:
len(target["fwr4"])

33

In [331]:
len(ref["fwr4"])

33

In [ ]:
TGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAG
TGGGGCCAGGGAACCCTGGTCACCGTCGCCTCA

In [312]:
from Bio.Seq import Seq

Seq("tggggccagggaaccctggtcaccgtcgcctca").translate()

Seq('WGQGTLVTVAS')

In [308]:
ref["sequence"]

'GGCCTCAGTGAAGGTCTCCTGCAAGGCATCTGGATACACCTTCACCAACCACTACATCCACTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAATAATCAACCCTAGTGGTGGTAGGACAAGTCACGTACAGGAGTTCCAGGGCAGAGTCACCATGACCAGGGACACGTCCACGAGTACTGTCTACATGGAGTTGAGTAGCCTGAGATCTGAGGACACGGCCGTGTACTTTTGTGCGAGACAACAGTGTAGTGCTTCCAACTGCTATGAGGAGAGTTTTGACCACTGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAGCCTCCACACAGAGCCCATCCGTCTTC'

In [298]:
target[check_these] == ref[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2     True
fwr3     True
cdr3     True
fwr4    False
Name: 1_705010661_igblastn_anarci_Heavy_IGHE_1125, dtype: bool

'ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCTGGATCCGCCAGCACCCAGGGAAGGGCCTGGAGTGGATTGGGTACATCTATTACAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTTACCATATCAGTAGACACGTCTAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACTGCCGCGGACACGGCCGTGTATTACTGTGCGANNNNNNNNNAACTCCNNNNTGATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCA'

In [193]:
spread.df_to_sheet(
    fillna(top_100_sadie_airr_only),
    sheet="sadie_airr_only",
    freeze_headers=True,
    freeze_index=True,
    replace=True,
)

<ipython-input-192-d3342252a2a0>:16: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  df.dtypes.apply(lambda x: x in ["float64", "Int16"])


In [213]:
spread.df_to_sheet(
    fillna(top_100_imgt_airr_only),
    sheet="imgt_airr_only",
    freeze_headers=True,
    freeze_index=True,
    replace=True,
)

<ipython-input-210-72c7d47bbfa2>:16: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  df.dtypes.apply(lambda x: x in ["float64", "Int16", "Int64"])


germline_alignment         ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCT...
germline_alignment_aa      TVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSR...
v_alignment_end                                                       316.00
v_germline_start                                                        1.00
v_germline_end                                                        316.00
v_identity                                                             82.22
d_alignment_start                                                     325.00
d_alignment_end                                                       334.00
d_germline_alignment                                              CAACTGGAAC
d_germline_alignment_aa                                                  QLE
d_germline_start                                                        5.00
d_germline_end                                                         14.00
d_identity                                                              0.00

## Try and find OAS similariities

I'm suspect of the entire airr file since it doesn't have fwr4

In [ ]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

In [ ]:
airr_dataframe = airr_api.run_dataframe(df_bz2, "index", "sequence")

In [ ]:
ref = df_bz2[df_bz2.columns.intersection(airr_dataframe.table.columns)].drop(
    "sequence_id", axis=1
)
target = airr_dataframe.table[
    airr_dataframe.table.columns.intersection(df_bz2.columns)
].drop("sequence_id", axis=1)

In [ ]:
ref.re

In [ ]:
for index in ref.index:
    sub_ref = ref.loc[index]
    sub_target = target.loc[index]
    aggree = sub_ref[sub_ref == sub_target].index
    disagree = sub_ref[sub_ref != sub_target].index
    if disagree.empty:
        print('yay')
    for dis_index in disagree:
        print(f"OAR-{dis_index}:{sub_ref[dis_index]}")
        print(f"Sadie-{dis_index}:{sub_target[dis_index]}\n")

    break

In [ ]:
disagree

In [ ]:
from pandas.testing import assert_series_equal

# Get IMGT airr

In [ ]:
imgt_airr = "../../tests/integration/airr/fixtures/OAS_airr_from_imgtvquest.tsv"
imgt_df = pd.read_csv(imgt_airr, delimiter="\t")

In [ ]:
spread.df_to_sheet(imgt_df, sheet="imgt_airr")